In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import xgboost as xgb

# Подготовка данных для обучения и тестирования модели:
from sklearn.model_selection import train_test_split

# Создание и обучение модели машинного обучения:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score

# Пример трансформации признаков
from sklearn.preprocessing import StandardScaler, PowerTransformer

# Пример ранней остановки для градиентного бустинга
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# Создание и обучение нейронной сети:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout


## Загрузка и обработка входных датасетов

In [2]:
# Загрузка данных из файла X_bp.xlsx с указанием кодировки в DataFrame: 'df1'
df1 = pd.read_excel('C:/Users/Shahm/DataspellProjects/VKR/Shakhmatov_VKR/X_bp.xlsx', engine='openpyxl')

# Вывод общего количества строк и столбцов
df1.shape

(1023, 11)

In [3]:
# Проверка на пропуски
df1.isnull().sum()

Unnamed: 0                              0
Соотношение матрица-наполнитель         0
Плотность, кг/м3                        0
модуль упругости, ГПа                   0
Количество отвердителя, м.%             0
Содержание эпоксидных групп,%_2         0
Температура вспышки, С_2                0
Поверхностная плотность, г/м2           0
Модуль упругости при растяжении, ГПа    0
Прочность при растяжении, МПа           0
Потребление смолы, г/м2                 0
dtype: int64

In [4]:
# Проверка на дубликаты
df1.duplicated().sum()

0

In [5]:
# Анализ данных с использованием описательной статистики
df1.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,1023.0,511.000000,295.458965,0.000000,255.500000,511.000000,766.500000,1022.000000
Соотношение матрица-наполнитель,1023.0,2.930366,0.913222,0.389403,2.317887,2.906878,3.552660,5.591742
"Плотность, кг/м3",1023.0,1975.734888,73.729231,1731.764635,1924.155467,1977.621657,2021.374375,2207.773481
"модуль упругости, ГПа",1023.0,739.923233,330.231581,2.436909,500.047452,739.664328,961.812526,1911.536477
"Количество отвердителя, м.%",1023.0,110.570769,28.295911,17.740275,92.443497,110.564840,129.730366,198.953207
"Содержание эпоксидных групп,%_2",1023.0,22.244390,2.406301,14.254985,20.608034,22.230744,23.961934,33.000000
"Температура вспышки, С_2",1023.0,285.882151,40.943260,100.000000,259.066528,285.896812,313.002106,413.273418
"Поверхностная плотность, г/м2",1023.0,482.731833,281.314690,0.603740,266.816645,451.864365,693.225017,1399.542362
"Модуль упругости при растяжении, ГПа",1023.0,73.328571,3.118983,64.054061,71.245018,73.268805,75.356612,82.682051
"Прочность при растяжении, МПа",1023.0,2466.922843,485.628006,1036.856605,2135.850448,2459.524526,2767.193119,3848.436732


In [6]:
# Смотрим первые 5 строк DataFrame: 'df1'
df1.head().T

,0,1,2,3,4
Unnamed: 0,0.000000,1.000000,2.000000,3.000000,4.000000
Соотношение матрица-наполнитель,1.857143,1.857143,1.857143,1.857143,2.771331
"Плотность, кг/м3",2030.000000,2030.000000,2030.000000,2030.000000,2030.000000
"модуль упругости, ГПа",738.736842,738.736842,738.736842,738.736842,753.000000
"Количество отвердителя, м.%",30.000000,50.000000,49.900000,129.000000,111.860000
"Содержание эпоксидных групп,%_2",22.267857,23.750000,33.000000,21.250000,22.267857
"Температура вспышки, С_2",100.000000,284.615385,284.615385,300.000000,284.615385
"Поверхностная плотность, г/м2",210.000000,210.000000,210.000000,210.000000,210.000000
"Модуль упругости при растяжении, ГПа",70.000000,70.000000,70.000000,70.000000,70.000000
"Прочность при растяжении, МПа",3000.000000,3000.000000,3000.000000,3000.000000,3000.000000


In [7]:
# Удаление столбца 'Unnamed: 0' с неинформативными данными
df1 = df1.drop(['Unnamed: 0'], axis=1)

# Вывод общего количества строк и столбцов
df1.shape

(1023, 10)

In [8]:
# Смотрим первые 5 строк DataFrame: 'df1'
df1.head().T

,0,1,2,3,4
Соотношение матрица-наполнитель,1.857143,1.857143,1.857143,1.857143,2.771331
"Плотность, кг/м3",2030.000000,2030.000000,2030.000000,2030.000000,2030.000000
"модуль упругости, ГПа",738.736842,738.736842,738.736842,738.736842,753.000000
"Количество отвердителя, м.%",30.000000,50.000000,49.900000,129.000000,111.860000
"Содержание эпоксидных групп,%_2",22.267857,23.750000,33.000000,21.250000,22.267857
"Температура вспышки, С_2",100.000000,284.615385,284.615385,300.000000,284.615385
"Поверхностная плотность, г/м2",210.000000,210.000000,210.000000,210.000000,210.000000
"Модуль упругости при растяжении, ГПа",70.000000,70.000000,70.000000,70.000000,70.000000
"Прочность при растяжении, МПа",3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
"Потребление смолы, г/м2",220.000000,220.000000,220.000000,220.000000,220.000000


In [9]:
# Загрузка данных из файла X_nup.xlsx с указанием кодировки в DataFrame: 'df2'
df2 = pd.read_excel('C:/Users/Shahm/DataspellProjects/VKR/Shakhmatov_VKR/X_nup.xlsx', engine='openpyxl')

# Вывод общего количества строк и столбцов
df2.shape

(1040, 4)

In [10]:
# Проверка на пропуски
df2.isnull().sum()

Unnamed: 0            0
Угол нашивки, град    0
Шаг нашивки           0
Плотность нашивки     0
dtype: int64

In [11]:
# Проверка на дубликаты
df2.duplicated().sum()

0

In [12]:
# Анализ данных с использованием описательной статистики
df2.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,1040.0,519.500000,300.366443,0.0,259.750000,519.500000,779.250000,1039.000000
"Угол нашивки, град",1040.0,45.000000,45.021650,0.0,0.000000,45.000000,90.000000,90.000000
Шаг нашивки,1040.0,6.911385,2.555181,0.0,5.102256,6.938000,8.587662,14.440522
Плотность нашивки,1040.0,57.248399,12.332438,0.0,49.970740,57.413594,65.107235,103.988901


In [13]:
# Смотрим первые 5 строк DataFrame: 'df2'
df2.head().T

,0,1,2,3,4
Unnamed: 0,0.0,1.0,2.0,3.0,4.0
"Угол нашивки, град",0.0,0.0,0.0,0.0,0.0
Шаг нашивки,4.0,4.0,4.0,5.0,5.0
Плотность нашивки,57.0,60.0,70.0,47.0,57.0


In [14]:
# Удаление столбца 'Unnamed: 0' с неинформативными данными
df2 = df2.drop(['Unnamed: 0'], axis=1)

# Вывод общего количества строк и столбцов
df2.shape

(1040, 3)

In [15]:
# Смотрим первые 5 строк DataFrame: 'df2'
df2.head().T

,0,1,2,3,4
"Угол нашивки, град",0.0,0.0,0.0,0.0,0.0
Шаг нашивки,4.0,4.0,4.0,5.0,5.0
Плотность нашивки,57.0,60.0,70.0,47.0,57.0


### Объединение данных 'df1' и 'df2' по индексу с использованием функции merge() с параметром 'inner'

In [16]:
# Объединение данных с использованием INNER
data = pd.merge(df1, df2, left_index=True, right_index=True, how='inner')

# Сохранение объединенных данных в новый xlsx-файл
data.to_excel('C:/Users/Shahm/DataspellProjects/VKR/Shakhmatov_VKR/data.xlsx', index=False, engine='openpyxl')

In [17]:
# Загрузка данных из файла data.xlsx с указанием кодировки в DataFrame: 'df'
df = pd.read_excel('C:/Users/Shahm/DataspellProjects/VKR/Shakhmatov_VKR/data.xlsx', engine='openpyxl')

# Приведем столбец "Угол нашивки" к значениям 0 и 1 и integer
df = df.replace({'Угол нашивки, град': {0.0 : 0, 90.0 : 1}})
df['Угол нашивки, град'] = df['Угол нашивки, град'].astype(int)

# Вывод общего количества строк и столбцов
df.shape

(1023, 13)

In [18]:
# Проверка на пропуски
df.isnull().sum()

Соотношение матрица-наполнитель         0
Плотность, кг/м3                        0
модуль упругости, ГПа                   0
Количество отвердителя, м.%             0
Содержание эпоксидных групп,%_2         0
Температура вспышки, С_2                0
Поверхностная плотность, г/м2           0
Модуль упругости при растяжении, ГПа    0
Прочность при растяжении, МПа           0
Потребление смолы, г/м2                 0
Угол нашивки, град                      0
Шаг нашивки                             0
Плотность нашивки                       0
dtype: int64

In [19]:
# Проверка на дубликаты
df.duplicated().sum()

0

In [20]:
#Поиск уникальных значений с помощью функции nunique
df.nunique()

Соотношение матрица-наполнитель         1014
Плотность, кг/м3                        1013
модуль упругости, ГПа                   1020
Количество отвердителя, м.%             1005
Содержание эпоксидных групп,%_2         1004
Температура вспышки, С_2                1003
Поверхностная плотность, г/м2           1004
Модуль упругости при растяжении, ГПа    1004
Прочность при растяжении, МПа           1004
Потребление смолы, г/м2                 1003
Угол нашивки, град                         2
Шаг нашивки                              989
Плотность нашивки                        988
dtype: int64

In [21]:
# Анализ данных с использованием описательной статистики
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Соотношение матрица-наполнитель,1023.0,2.930366,0.913222,0.389403,2.317887,2.906878,3.552660,5.591742
"Плотность, кг/м3",1023.0,1975.734888,73.729231,1731.764635,1924.155467,1977.621657,2021.374375,2207.773481
"модуль упругости, ГПа",1023.0,739.923233,330.231581,2.436909,500.047452,739.664328,961.812526,1911.536477
"Количество отвердителя, м.%",1023.0,110.570769,28.295911,17.740275,92.443497,110.564840,129.730366,198.953207
"Содержание эпоксидных групп,%_2",1023.0,22.244390,2.406301,14.254985,20.608034,22.230744,23.961934,33.000000
"Температура вспышки, С_2",1023.0,285.882151,40.943260,100.000000,259.066528,285.896812,313.002106,413.273418
"Поверхностная плотность, г/м2",1023.0,482.731833,281.314690,0.603740,266.816645,451.864365,693.225017,1399.542362
"Модуль упругости при растяжении, ГПа",1023.0,73.328571,3.118983,64.054061,71.245018,73.268805,75.356612,82.682051
"Прочность при растяжении, МПа",1023.0,2466.922843,485.628006,1036.856605,2135.850448,2459.524526,2767.193119,3848.436732
"Потребление смолы, г/м2",1023.0,218.423144,59.735931,33.803026,179.627520,219.198882,257.481724,414.590628


In [22]:
# Смотрим первые 5 строк DataFrame: 'df'
df.head().T

,0,1,2,3,4
Соотношение матрица-наполнитель,1.857143,1.857143,1.857143,1.857143,2.771331
"Плотность, кг/м3",2030.000000,2030.000000,2030.000000,2030.000000,2030.000000
"модуль упругости, ГПа",738.736842,738.736842,738.736842,738.736842,753.000000
"Количество отвердителя, м.%",30.000000,50.000000,49.900000,129.000000,111.860000
"Содержание эпоксидных групп,%_2",22.267857,23.750000,33.000000,21.250000,22.267857
"Температура вспышки, С_2",100.000000,284.615385,284.615385,300.000000,284.615385
"Поверхностная плотность, г/м2",210.000000,210.000000,210.000000,210.000000,210.000000
"Модуль упругости при растяжении, ГПа",70.000000,70.000000,70.000000,70.000000,70.000000
"Прочность при растяжении, МПа",3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
"Потребление смолы, г/м2",220.000000,220.000000,220.000000,220.000000,220.000000


## Подготовка данных для обучения и тестирования модели:

In [23]:
# Выделение признаков и целевых переменных
X = df.drop(['Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа'], axis=1)
y1 = df['Модуль упругости при растяжении, ГПа']
y2 = df['Прочность при растяжении, МПа']

In [24]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=42)
_, _, y2_train, y2_test = train_test_split(X, y2, test_size=0.2, random_state=42)

## Создание и обучение модели машинного обучения:

In [25]:
# Обучение модели случайного леса для Модуля упругости при растяжении
rf_model1 = RandomForestRegressor(random_state=42)
rf_model1.fit(X_train, y1_train)

RandomForestRegressor(random_state=42)

In [26]:
# Предсказание и оценка качества модели
y1_pred = rf_model1.predict(X_test)
print("Метрики для Модуля упругости при растяжении:")
print("MAE:", mean_absolute_error(y1_test, y1_pred))
print("MSE:", mean_squared_error(y1_test, y1_pred))
print("R^2:", r2_score(y1_test, y1_pred))

Метрики для Модуля упругости при растяжении:
MAE: 2.5311593606255163
MSE: 10.216291862364052
R^2: -0.09027102564520129


In [27]:
# Обучение модели случайного леса для Прочности при растяжении
rf_model2 = RandomForestRegressor(random_state=42)
rf_model2.fit(X_train, y2_train)

RandomForestRegressor(random_state=42)

In [28]:
# Предсказание и оценка качества модели
y2_pred = rf_model2.predict(X_test)
# Метрики для Прочности при растяжении:
print("MAE:", mean_absolute_error(y2_test, y2_pred))
print("MSE:", mean_squared_error(y2_test, y2_pred))
print("R^2:", r2_score(y2_test, y2_pred))

MAE: 373.4496910619894
MSE: 227667.93998074872
R^2: -0.04940156550370145


## Создание и обучение нейронной сети:

In [29]:
# Создание архитектуры нейронной сети
def create_model():
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='linear'))

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [30]:
# Обучение нейронной сети для Модуля упругости при растяжении
nn_model1 = create_model()
nn_model1.fit(X_train, y1_train, epochs=100, batch_size=32,verbose=1, validation_split=0.2)

Epoch 1/100
21/21 [==============================] - 1s 14ms/step - loss: 39635.4531 - val_loss: 412.7708
Epoch 2/100
21/21 [==============================] - 0s 6ms/step - loss: 7464.2637 - val_loss: 985.7695
Epoch 3/100
21/21 [==============================] - 0s 6ms/step - loss: 4648.2612 - val_loss: 100.6798
Epoch 4/100
21/21 [==============================] - 0s 6ms/step - loss: 2891.8887 - val_loss: 84.6644
Epoch 5/100
21/21 [==============================] - 0s 6ms/step - loss: 2783.8447 - val_loss: 141.2821
Epoch 6/100
21/21 [==============================] - 0s 6ms/step - loss: 1697.4916 - val_loss: 80.8750
Epoch 7/100
21/21 [==============================] - 0s 6ms/step - loss: 1313.7982 - val_loss: 127.6008
Epoch 8/100
21/21 [==============================] - 0s 6ms/step - loss: 1124.3647 - val_loss: 158.0049
Epoch 9/100
21/21 [==============================] - 0s 6ms/step - loss: 788.9846 - val_loss: 138.3905
Epoch 10/100
21/21 [==============================] - 0s 6ms/step

In [31]:
# Предсказание и оценка качества нейронной сети
y1_nn_pred = nn_model1.predict(X_test)
print("Метрики для Модуля упругости при растяжении (нейронная сеть):")
print("MAE:", mean_absolute_error(y1_test, y1_nn_pred))
print("MSE:", mean_squared_error(y1_test, y1_nn_pred))
print("R^2:", r2_score(y1_test, y1_nn_pred))

7/7 [==============================] - 0s 2ms/step
Метрики для Модуля упругости при растяжении (нейронная сеть):
MAE: 7.781325021004784
MSE: 77.59835096621428
R^2: -7.2812075884384075


In [32]:
# Обучение нейронной сети для Прочности при растяжении
nn_model2 = create_model()
nn_model2.fit(X_train, y2_train, epochs=100, batch_size=32, verbose=1, validation_split=0.2)

Epoch 1/100
21/21 [==============================] - 1s 13ms/step - loss: 3160570.5000 - val_loss: 512457.3125
Epoch 2/100
21/21 [==============================] - 0s 6ms/step - loss: 427267.9688 - val_loss: 321182.2812
Epoch 3/100
21/21 [==============================] - 0s 6ms/step - loss: 335909.7188 - val_loss: 288358.2812
Epoch 4/100
21/21 [==============================] - 0s 6ms/step - loss: 309428.8438 - val_loss: 272740.0312
Epoch 5/100
21/21 [==============================] - 0s 6ms/step - loss: 301429.7812 - val_loss: 265798.5625
Epoch 6/100
21/21 [==============================] - 0s 6ms/step - loss: 296634.1562 - val_loss: 265129.1562
Epoch 7/100
21/21 [==============================] - 0s 6ms/step - loss: 302350.1562 - val_loss: 262597.6562
Epoch 8/100
21/21 [==============================] - 0s 6ms/step - loss: 306331.8125 - val_loss: 263318.8125
Epoch 9/100
21/21 [==============================] - 0s 6ms/step - loss: 287956.7188 - val_loss: 270346.0625
Epoch 10/100
21/2

In [33]:
# Предсказание и оценка качества нейронной сети
y2_nn_pred = nn_model2.predict(X_test)
print("\nМетрики для Прочности при растяжении (нейронная сеть):")
print("MAE:", mean_absolute_error(y2_test, y2_nn_pred))
print("MSE:", mean_squared_error(y2_test, y2_nn_pred))
print("R^2:", r2_score(y2_test, y2_nn_pred))

7/7 [==============================] - 0s 2ms/step

Метрики для Прочности при растяжении (нейронная сеть):
MAE: 419.01490367132936
MSE: 281313.7833520853
R^2: -0.29667411525931175


## Создание рекомендательной системы:

In [34]:
print(X.columns)

Index(['Соотношение матрица-наполнитель', 'Плотность, кг/м3',
       'модуль упругости, ГПа', 'Количество отвердителя, м.%',
       'Содержание эпоксидных групп,%_2', 'Температура вспышки, С_2',
       'Поверхностная плотность, г/м2', 'Потребление смолы, г/м2',
       'Угол нашивки, град', 'Шаг нашивки', 'Плотность нашивки'],
      dtype='object')


In [35]:
# В этом случае рекомендательная система может быть основана на предсказаниях нейронной сети. Рекомендации могут быть сформированы путем определения оптимального соотношения матрица-наполнитель, которое минимизирует ошибку предсказания или максимизирует определенные характеристики материала.
def recommend_ratio(nn_model1, nn_model2, X_sample):
    # Входные данные должны быть предварительно масштабированы и нормализованы
    best_ratio = None
    best_error = float('inf')

    for ratio in np.linspace(0.1, 1, num=10):
        X_sample_modified = X_sample.copy()
        X_sample_modified[0] = ratio
        pred1 = nn_model1.predict(np.array([X_sample_modified]))[0]
        pred2 = nn_model2.predict(np.array([X_sample_modified]))[0]

        # Определение средней ошибки предсказания
        error = (abs(pred1 - y1_test.iloc[0]) + abs(pred2 - y2_test.iloc[0])) / 2

        if error < best_error:
            best_error = error
            best_ratio = ratio

    return best_ratio

In [36]:

X_sample = X_test.iloc[0]
best_ratio = recommend_ratio(nn_model1, nn_model2, X_sample)
print("Рекомендуемое соотношение матрица-наполнитель:", best_ratio)

1/1 [==============================] - 0s 23ms/step
Рекомендуемое соотношение матрица-наполнитель: 1.0


In [37]:
# Обучение StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

# Масштабирование данных
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [38]:

X_sample = X_test_scaled[0, :]
best_ratio = recommend_ratio(nn_model1, nn_model2, X_sample)
print("Рекомендуемое соотношение матрица-наполнитель:", best_ratio)

1/1 [==============================] - 0s 24ms/step
Рекомендуемое соотношение матрица-наполнитель: 1.0


In [39]:
def print_metrics(y_true, y_pred, label):
    print(f"Метрики для {label}:")
    print("MAE:", mean_absolute_error(y_true, y_pred))
    print("MSE:", mean_squared_error(y_true, y_pred))
    print("R^2:", r2_score(y_true, y_pred))

xgb_model1 = xgb.XGBRegressor(random_state=42)
xgb_model1.fit(X_train_scaled, y1_train)
y1_xgb_pred = xgb_model1.predict(X_test_scaled)

xgb_model2 = xgb.XGBRegressor(random_state=42)
xgb_model2.fit(X_train_scaled, y2_train)
y2_xgb_pred = xgb_model2.predict(X_test_scaled)

print_metrics(y1_test, y1_xgb_pred, "Модуль упругости при растяжении (XGBoost)")
print_metrics(y2_test, y2_xgb_pred, "Прочность при растяжении (XGBoost)")


Метрики для Модуль упругости при растяжении (XGBoost):
MAE: 2.8464042755061345
MSE: 12.676503683484464
R^2: -0.3528220276774312
Метрики для Прочность при растяжении (XGBoost):
MAE: 427.7667023584024
MSE: 301013.0903852201
R^2: -0.38747514610123757
